Benchmarking on One Machine 
=========


**Typical User Experience**

Laptop Specs:
    
    Intel Core i7
    16gb RAM
    NVIDIA GeForce GTX 965M 2GB GDDR5 memory 
    Microsoft Windows 10
    Running Jupyter Notebooks and multiple programs in the background

In [1]:
from IPython.display import HTML

In [2]:
HTML('''<script>
  function code_toggle() {
    if (code_shown){
      $('div.input').hide('500');
      $('#toggleButton').val('Show Code')
    } else {
      $('div.input').show('500');
      $('#toggleButton').val('Hide Code')
    }
    code_shown = !code_shown
  }

  $( document ).ready(function(){
    code_shown=false;
    $('div.input').hide()
  });
</script>
<form action="javascript:code_toggle()"><input type="submit" id="toggleButton" value="Show Code"></form>''')

**Import Packages**

In [1]:
import warnings
warnings.simplefilter(action='ignore')

import pandas as pd
import numpy as np
import re
import os

import time #cpu time
import psutil #memory usage
#tensorflow
import tensorflow as tf

#Scikitlearn
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_svmlight_file
from scipy.sparse import coo_matrix,csr_matrix,lil_matrix
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

# For Visualization
import matplotlib.pyplot as plt
#displays better in jupyter notebooks
%matplotlib inline

In [2]:
#Download data 
#!wget https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/binary/a1a.t

In [3]:
def get_data(file):
    data = load_svmlight_file(file)#avazu-app.tr.bz2
    return data[0], data[1]

In [4]:
mem_baseline=psutil.virtual_memory() #  physical memory usage
print('Here is the memory baseline before importing data:\n',mem_baseline)

Here is the memory baseline before importing data:
 svmem(total=17101512704, available=9854275584, percent=42.4, used=7247237120, free=9854275584)


In [5]:
X, y =get_data('a1a.t') #import raw data
mem_InData=psutil.virtual_memory()

In [6]:
print('Here is the memory usage after importing data:\n',mem_InData) #  physical memory usage
print('\nThe time took to import the raw data:')
exec_time1 = %%timeit -o X, y =get_data('a1a.t') #import raw data

Here is the memory usage after importing data:
 svmem(total=17101512704, available=9848799232, percent=42.4, used=7252713472, free=9848799232)

The time took to import the raw data:
248 ms ± 26.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
#%%timeit
#X, y =get_data('a1a.t.bz2') #import binary data

In [8]:
print('Here is the sparse matrix format for the data:') 
X

Here is the sparse matrix format for the data:


<30956x123 sparse matrix of type '<class 'numpy.float64'>'
	with 429343 stored elements in Compressed Sparse Row format>

In [9]:
#May not be appropriate for time dependant data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

Execute Logistic Regression in SciKit Learn
-------

In [10]:
# instantiate a logistic regression model, and fit with X and y
# Here are the 
model = linear_model.LogisticRegression(penalty='l2',\
                                        C=1.0,\
                                        tol=0.0001,\
                                        fit_intercept=True,\
                                        n_jobs=1,\
                                        max_iter=100)
start = time.time()
model = model.fit(X, y.ravel())
end = time.time()

In [11]:
exec_time2=(end - start)
print('The time taken to execute the logistic regression:',exec_time2,'seconds')

The time taken to execute the logistic regression: 0.5869946479797363 seconds


In [12]:
y_pred = model.predict(X_test)
y_obs = y_test
y_score = y_pred

**Model Metrics**

In [13]:
r2=metrics.r2_score(y_obs, y_pred)
accuracy=model.score(X_train, y_train)
prec=metrics.precision_score(y_obs, y_pred, labels=None, pos_label=1)
recall=metrics.recall_score(y_obs, y_pred, labels=None, pos_label=1)
f1 = metrics.f1_score(y_obs,y_pred)
ROC_AUC = metrics.roc_auc_score(y_obs, y_score)
print('The correlation coefficient:',r2,\
      '\nThe accuracy of the model:',accuracy,\
      '\nThe precision (tp / (tp + fp)):',prec,\
      '\nThe recall (tp / (tp + fn)):',recall,\
      '\nThe f1 score is:',f1,\
      '\nThe Area Under the Curve score is:',ROC_AUC)

The correlation coefficient: 0.1588200017358622 
The accuracy of the model: 0.8501928640308583 
The precision (tp / (tp + fp)): 0.7191295546558705 
The recall (tp / (tp + fn)): 0.5874328234807772 
The f1 score is: 0.6466439135381115 
The Area Under the Curve score is: 0.7581257999666293


Execute Logistic Regression in Tensorflow
-------------

Here the GPU is used for the task but restricted to using only 75% of GPU resources in order to prevent crashing.

In [912]:
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.75)

In [1059]:
learning_rate =  0.01
max_iter = 10
batch_size = 30

train_file = os.getcwd()+'\\tensorflow-models\\data\\libsvm_data\\train.data'
test_file = os.getcwd()+'\\tensorflow-models\\data\\libsvm_data\\test.data'

In [1060]:
#len(feature_ids)

In [1074]:
class DataSet(object):
    
    def __init__(self):
        self.iter = 0
        self.epoch_pass = 0

    def load(self, file):
        '''
        '''
        self.ins_num = 0 #<set at zero
        f = open(file, "r")
        self.y = []
        self.feature_ids = []
        self.feature_values = []
        self.ins_feature_interval = []
        self.max_ins_feature_interval = []
        self.ins_feature_interval.append(0)#makes zero the starting value in ins_feature_interval
        self.max_token=[]
        for line in f.readlines():#iterating through open file
            regexp = re.compile(r':')#<---If feature has a colon then do
            tokens = line.split(" ")#split lines in the file
            #tokens.remove('\n')
            #print(tokens)
            self.y.append(float(tokens[0]))#append to y the first value in tokens (+1,-1,1)
            try:
                tokens[-1] = tokens[-1].strip()#<----remove '\n'
                tokens.remove('') #<---remove '' empty in list
            except:
                pass
            
            #                         last value in list is that value + (line splits -1)<--maybe adjusting for return (\n) or y value 
            #print(self.ins_feature_interval[-1]+ len(tokens)-1)#<----stacks the batch sizes
            self.ins_feature_interval.append(self.ins_feature_interval[-1]+ len(tokens)-1)
            #print(len(self.ins_feature_interval))
            for feature in tokens:#(len(tokens)~16
                #print(feature)
                if regexp.search(feature):#if there is a colon in feature
                    self.max_token.append(feature)
                    feature_id, feature_value = feature.split(":") #split on colon
                    if int(feature_id):#if the feature id is integer
                        self.feature_ids.append(int(feature_id))#append to feature ids
                        self.feature_values.append(float(feature_value)) #append feature values
                    
            self.ins_num += 1 #set ins_num to 1
        self.feature_num=max(self.feature_ids)#modify feature_num to max of ids (maximum # of features)
        #self.max_ins_feature_interval=max(self.ins_feature_interval)
        #print(self.max_ins_feature_interval)
        print('the max number of features:',self.feature_num)

    def mini_batch(self, batch_size):
        begin = self.iter #begins as 0 as defined above
        end = self.iter #starts with 0 as defined above
        if self.iter + batch_size > self.ins_num: #if 0 + batchsize(10) > ins_num(1) defined in def load
            end = self.ins_num #set end to be ins_num(1) 
            self.iter = 0 #set iter to 0
            self.epoch_pass += 1 #add +1 to epoch_pass 
        else:
            end += batch_size#add batch size to end, which should be equal to batch size
            #print('end:',end)
            #print('batch_size:',batch_size)
            self.iter = end#set self.iter to batch size
            #(begin, end)setting bounds moving across batch length in data
            #print((begin, end))#slicing action of data (0, 15) (15, 30) (30, 45) (60,...
        return self.slice(begin, end)
 
    def slice(self, begin, end):
        sparse_index = []
        sparse_ids = []
        sparse_values = []
        sparse_shape = []
        max_feature_num = 0
        for i in range(begin, end):#within range begin, end
            #                     15,461,906,1351                  0,446,891,1338 =~15 supposed to be length of token
            #                   (token length + range number +1) - (token length + range number)
            feature_num_range = self.ins_feature_interval[i + 1] - self.ins_feature_interval[i]
            #print(feature_num_range)
            if feature_num_range > max_feature_num:   #<---if above feature_num_range is larger adjust max_feature_num 
                max_feature_num = feature_num_range
                #       0,446,891,1338                  15,461,906,1351
                #(token length + range number) ,       (token length + range number +1)
            #self.max_ins_feature_interval-len(self.feature_ids)<-------------------
            #print(self.ins_feature_interval[i],self.ins_feature_interval[i + 1])
            #print(self.feature_ids[i])
            for j in range(self.ins_feature_interval[i], self.ins_feature_interval[i + 1]):
                #print(j)   
                print(j,(len(self.feature_ids)))
                #print(self.ins_feature_interval[i + 1])
                #print(range(self.ins_feature_interval[i], self.ins_feature_interval[i + 1]))
                #15 vals:                  0  , 0-14,446-460,891-905
                sparse_index.append([i - begin, j - self.ins_feature_interval[i]]) # index must be accent<----
                #[0, 0]-[0, 14][0, 0]-[0, 12]
                #print([i - begin, j - self.ins_feature_interval[i]])
                sparse_ids.append(self.feature_ids[j])#<---------------
                #print(self.feature_ids[j])
                sparse_values.append(self.feature_values[j])#<-----
                #print(self.feature_values[j])

            sparse_shape.append(end - begin)
            #print(end - begin)#<-----30
            sparse_shape.append(max_feature_num)
            #print(max_feature_num)#<-----15
            #       Creates array shape of 30,1 of y values  (30, 1)
            y = np.array(self.y[begin:end]).reshape((end - begin, 1))
            #begin:0,30,60,90,120,150,180,210 intervals of 30
            #end: 30,60,90,120,150,180,210,240
            #            0            0                       0                 30            30
            print(len(sparse_index), len(sparse_ids), len(sparse_values), len(sparse_shape), len(y))
            return (sparse_index, sparse_ids, sparse_values, sparse_shape, y)

In [1075]:
class BinaryLogisticRegression(object):
    def __init__(self, feature_num):
        self.feature_num = feature_num
        self.sparse_index = tf.placeholder(tf.int64)
        self.sparse_ids = tf.placeholder(tf.int64)
        self.sparse_values = tf.placeholder(tf.float32)
        self.sparse_shape = tf.placeholder(tf.int64)
        self.w = tf.Variable(tf.random_normal([self.feature_num, 1], stddev=0.1))
        self.y = tf.placeholder("float", [None, 1])

    def forward(self):
        return tf.nn.embedding_lookup_sparse(self.w,
                                             tf.SparseTensor(self.sparse_index, self.sparse_ids, self.sparse_shape),
                                             tf.SparseTensor(self.sparse_index, self.sparse_values, self.sparse_shape),
                                             combiner="sum")

In [1076]:
train_set = DataSet()
train_set.load(train_file)
feature_num=train_set.feature_num

the max number of features: 123


In [1077]:
#len(train_set.max_token)

In [1078]:
#len(train_set.feature_ids)

In [1079]:
test_set = DataSet()
test_set.load(test_file)

the max number of features: 122


In [1080]:
model = BinaryLogisticRegression(feature_num)

In [1081]:
y = model.forward()

In [1082]:
loss = tf.reduce_sum(tf.nn.sigmoid_cross_entropy_with_logits(labels=y, logits=model.y))

In [1083]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

In [1084]:
probability_output = tf.nn.sigmoid(y)

In [1085]:
session = tf.Session()
init_all_variable = tf.global_variables_initializer()
init_local_variable = tf.local_variables_initializer()
session.run([init_all_variable, init_local_variable])

[None, None]

In [1086]:
while train_set.epoch_pass < max_iter:
    sparse_index, sparse_ids, sparse_values, sparse_shape, mb_y = train_set.mini_batch(batch_size)
    _, loss_, prob_out = session.run([optimizer, loss, probability_output],
                                     feed_dict={model.sparse_index: sparse_index,
                                                model.sparse_ids: sparse_ids,
                                                model.sparse_values: sparse_values,
                                                model.sparse_shape: sparse_shape,
                                                model.y: mb_y})

auc = roc_auc_score(mb_y, prob_out)
print("epoch: ", train_set.epoch_pass, " auc: ", auc)

0 314815
1 314815
2 314815
3 314815
4 314815
5 314815
6 314815
7 314815
8 314815
9 314815
10 314815
11 314815
12 314815
13 314815
14 14 14 2 30
416 314815
417 314815
418 314815
419 314815
420 314815
421 314815
422 314815
423 314815
424 314815
425 314815
426 314815
427 314815
428 314815
429 314815
14 14 14 2 30
831 314815
832 314815
833 314815
834 314815
835 314815
836 314815
837 314815
838 314815
839 314815
840 314815
841 314815
842 314815
843 314815
844 314815
14 14 14 2 30
1248 314815
1249 314815
1250 314815
1251 314815
1252 314815
1253 314815
1254 314815
1255 314815
1256 314815
1257 314815
1258 314815
1259 314815
12 12 12 2 30
1660 314815
1661 314815
1662 314815
1663 314815
1664 314815
1665 314815
1666 314815
1667 314815
1668 314815
1669 314815
1670 314815
1671 314815
1672 314815
1673 314815
14 14 14 2 30
2076 314815
2077 314815
2078 314815
2079 314815
2080 314815
2081 314815
2082 314815
2083 314815
2084 314815
2085 314815
2086 314815
2087 314815
2088 314815
2089 314815
14 14 14 2 3

20779 314815
20780 314815
20781 314815
20782 314815
20783 314815
20784 314815
20785 314815
20786 314815
20787 314815
20788 314815
20789 314815
20790 314815
20791 314815
20792 314815
14 14 14 2 30
21196 314815
21197 314815
21198 314815
21199 314815
21200 314815
21201 314815
21202 314815
21203 314815
21204 314815
21205 314815
21206 314815
21207 314815
21208 314815
21209 314815
14 14 14 2 30
21612 314815
21613 314815
21614 314815
21615 314815
21616 314815
21617 314815
21618 314815
21619 314815
21620 314815
21621 314815
21622 314815
21623 314815
21624 314815
21625 314815
14 14 14 2 30
22032 314815
22033 314815
22034 314815
22035 314815
22036 314815
22037 314815
22038 314815
22039 314815
22040 314815
22041 314815
22042 314815
22043 314815
22044 314815
22045 314815
14 14 14 2 30
22448 314815
22449 314815
22450 314815
22451 314815
22452 314815
22453 314815
22454 314815
22455 314815
22456 314815
22457 314815
22458 314815
22459 314815
22460 314815
22461 314815
14 14 14 2 30
22858 314815
22859 3

41133 314815
41134 314815
41135 314815
41136 314815
41137 314815
41138 314815
41139 314815
41140 314815
41141 314815
41142 314815
41143 314815
41144 314815
41145 314815
41146 314815
14 14 14 2 30
41549 314815
41550 314815
41551 314815
41552 314815
41553 314815
41554 314815
41555 314815
41556 314815
41557 314815
41558 314815
41559 314815
41560 314815
41561 314815
41562 314815
14 14 14 2 30
41966 314815
41967 314815
41968 314815
41969 314815
41970 314815
41971 314815
41972 314815
41973 314815
41974 314815
41975 314815
41976 314815
41977 314815
41978 314815
41979 314815
14 14 14 2 30
42384 314815
42385 314815
42386 314815
42387 314815
42388 314815
42389 314815
42390 314815
42391 314815
42392 314815
42393 314815
42394 314815
42395 314815
42396 314815
42397 314815
14 14 14 2 30
42799 314815
42800 314815
42801 314815
42802 314815
42803 314815
42804 314815
42805 314815
42806 314815
42807 314815
42808 314815
42809 314815
42810 314815
42811 314815
42812 314815
14 14 14 2 30
43210 314815
43211 3

59447 314815
59448 314815
59449 314815
59450 314815
59451 314815
59452 314815
59453 314815
59454 314815
59455 314815
59456 314815
59457 314815
59458 314815
59459 314815
59460 314815
14 14 14 2 30
59866 314815
59867 314815
59868 314815
59869 314815
59870 314815
59871 314815
59872 314815
59873 314815
59874 314815
59875 314815
59876 314815
59877 314815
59878 314815
59879 314815
14 14 14 2 30
60283 314815
60284 314815
60285 314815
60286 314815
60287 314815
60288 314815
60289 314815
60290 314815
60291 314815
60292 314815
60293 314815
60294 314815
60295 314815
60296 314815
14 14 14 2 30
60699 314815
60700 314815
60701 314815
60702 314815
60703 314815
60704 314815
60705 314815
60706 314815
60707 314815
60708 314815
60709 314815
60710 314815
60711 314815
60712 314815
14 14 14 2 30
61116 314815
61117 314815
61118 314815
61119 314815
61120 314815
61121 314815
61122 314815
61123 314815
61124 314815
61125 314815
61126 314815
61127 314815
61128 314815
61129 314815
14 14 14 2 30
61531 314815
61532 3

80698 314815
80699 314815
80700 314815
80701 314815
80702 314815
80703 314815
80704 314815
80705 314815
80706 314815
80707 314815
80708 314815
14 14 14 2 30
81111 314815
81112 314815
81113 314815
81114 314815
81115 314815
81116 314815
81117 314815
81118 314815
81119 314815
81120 314815
81121 314815
81122 314815
81123 314815
81124 314815
14 14 14 2 30
81529 314815
81530 314815
81531 314815
81532 314815
81533 314815
81534 314815
81535 314815
81536 314815
81537 314815
81538 314815
81539 314815
81540 314815
81541 314815
81542 314815
14 14 14 2 30
81945 314815
81946 314815
81947 314815
81948 314815
81949 314815
81950 314815
81951 314815
81952 314815
81953 314815
81954 314815
81955 314815
81956 314815
81957 314815
81958 314815
14 14 14 2 30
82365 314815
82366 314815
82367 314815
82368 314815
82369 314815
82370 314815
82371 314815
82372 314815
82373 314815
82374 314815
82375 314815
82376 314815
82377 314815
82378 314815
14 14 14 2 30
82779 314815
82780 314815
82781 314815
82782 314815
82783 3

...

14 14 14 2 30
249693 314815
249694 314815
249695 314815
249696 314815
249697 314815
249698 314815
249699 314815
249700 314815
249701 314815
249702 314815
249703 314815
249704 314815
249705 314815
249706 314815
14 14 14 2 30
250105 314815
250106 314815
250107 314815
250108 314815
250109 314815
250110 314815
250111 314815
250112 314815
250113 314815
250114 314815
250115 314815
250116 314815
250117 314815
250118 314815
14 14 14 2 30
250520 314815
250521 314815
250522 314815
250523 314815
250524 314815
250525 314815
250526 314815
250527 314815
250528 314815
250529 314815
250530 314815
250531 314815
250532 314815
250533 314815
14 14 14 2 30
250937 314815
250938 314815
250939 314815
250940 314815
250941 314815
250942 314815
250943 314815
250944 314815
250945 314815
250946 314815
250947 314815
250948 314815
250949 314815
250950 314815
14 14 14 2 30
251351 314815
251352 314815
251353 314815
251354 314815
251355 314815
251356 314815
251357 314815
251358 314815
251359 314815
251360 314815
251361

14 14 14 2 30
275080 314815
275081 314815
275082 314815
275083 314815
275084 314815
275085 314815
275086 314815
275087 314815
275088 314815
275089 314815
275090 314815
275091 314815
275092 314815
275093 314815
14 14 14 2 30
275495 314815
275496 314815
275497 314815
275498 314815
275499 314815
275500 314815
275501 314815
275502 314815
275503 314815
275504 314815
275505 314815
275506 314815
275507 314815
275508 314815
14 14 14 2 30
275913 314815
275914 314815
275915 314815
275916 314815
275917 314815
275918 314815
275919 314815
275920 314815
275921 314815
275922 314815
275923 314815
275924 314815
275925 314815
275926 314815
14 14 14 2 30
276331 314815
276332 314815
276333 314815
276334 314815
276335 314815
276336 314815
276337 314815
276338 314815
276339 314815
276340 314815
276341 314815
276342 314815
276343 314815
276344 314815
14 14 14 2 30
276746 314815
276747 314815
276748 314815
276749 314815
276750 314815
276751 314815
276752 314815
276753 314815
276754 314815
276755 314815
276756

293814 314815
293815 314815
293816 314815
293817 314815
293818 314815
293819 314815
14 14 14 2 30
294218 314815
294219 314815
294220 314815
294221 314815
294222 314815
294223 314815
294224 314815
294225 314815
294226 314815
294227 314815
294228 314815
294229 314815
294230 314815
294231 314815
14 14 14 2 30
294634 314815
294635 314815
294636 314815
294637 314815
294638 314815
294639 314815
294640 314815
294641 314815
294642 314815
294643 314815
294644 314815
294645 314815
294646 314815
294647 314815
14 14 14 2 30
295052 314815
295053 314815
295054 314815
295055 314815
295056 314815
295057 314815
295058 314815
295059 314815
295060 314815
295061 314815
295062 314815
295063 314815
295064 314815
295065 314815
14 14 14 2 30
295472 314815
295473 314815
295474 314815
295475 314815
295476 314815
295477 314815
295478 314815
295479 314815
295480 314815
295481 314815
295482 314815
295483 314815
295484 314815
295485 314815
14 14 14 2 30
295885 314815
295886 314815
295887 314815
295888 314815
295889

311269 314815
311270 314815
311271 314815
311272 314815
311273 314815
311274 314815
311275 314815
311276 314815
311277 314815
311278 314815
311279 314815
311280 314815
311281 314815
311282 314815
14 14 14 2 30
311686 314815
311687 314815
311688 314815
311689 314815
311690 314815
311691 314815
311692 314815
311693 314815
311694 314815
311695 314815
311696 314815
311697 314815
311698 314815
311699 314815
14 14 14 2 30
312102 314815
312103 314815
312104 314815
312105 314815
312106 314815
312107 314815
312108 314815
312109 314815
312110 314815
312111 314815
312112 314815
312113 314815
312114 314815
312115 314815
14 14 14 2 30
312515 314815
312516 314815
312517 314815
312518 314815
312519 314815
312520 314815
312521 314815
312522 314815
312523 314815
312524 314815
312525 314815
312526 314815
312527 314815
312528 314815
14 14 14 2 30
312935 314815
312936 314815
312937 314815
312938 314815
312939 314815
312940 314815
312941 314815
312942 314815
312943 314815
312944 314815
312945 314815
312946

ValueError: Found input variables with inconsistent numbers of samples: [16, 1]

#for loop to create our own timeit

range(314539, 314554)

314986

range(314986, 315001)

length of J 314815

314986 ins_feature_interval

print(j,(len(self.feature_ids)))

314553 314815

15 15 15 2 30

314986 314815